# Create simple chloropleth maps by ecoregion, state, and watershed

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib


## Function for making maps

In [ ]:
def make_map(df, shape, merge_column, title, density=True, log=True, axs=[]):
    df = df.assign(nonzero=df['area']>0)
    df = df.groupby('reg').sum()
    df['area'] = df['area']*0.01

    shape[merge_column] = shape[merge_column].astype(int)
    shape = shape.merge(df, left_on=merge_column, right_on='reg')
    
    if density:
        # Calculate density in ha/ha or count/ha
        shape['area'] = shape['area']/(0.01*shape.area/(0.000089831528412**2))
        shape['nonzero'] = shape['nonzero']/(0.01*shape.area/(0.000089831528412**2))
    # Make plots
    if len(axs) == 0:
        fig, axs = plt.subplots(1, 2, figsize=[12,6])
    if log:
        shape.plot(column='area', ax=axs[0], legend=True,
                   norm=matplotlib.colors.LogNorm(vmin=shape.loc[shape['area']>0,'area'].min(), vmax=shape['area'].max()))
    else:
        shape.plot(column='area', ax=axs[0], legend=True)
    shape.boundary.plot(ax=axs[0], linewidth=0.25)
    axs[0].set_title(title+' Reservoir Area Density\n(Reservoir Area Fraction)', size=15)
    axs[0].set_ylabel('Latitude', size=14)
    axs[0].set_xlabel('Longitude', size=14)
    axs[0].tick_params(axis='both', labelsize=12 )
    if log:    
        shape.plot(column='nonzero', ax=axs[1], legend=True,
                  norm=matplotlib.colors.LogNorm(vmin=shape.loc[shape['nonzero']>0,'nonzero'].min(), vmax=shape['nonzero'].max()))
    else:
        shape.plot(column='nonzero', ax=axs[1], legend=True)
    shape.boundary.plot(ax=axs[1], linewidth=0.25)
    axs[1].set_title(title+' Reservoir Count Density\n(Reservoirs per ha)', size=15)
    axs[1].set_ylabel('Latitude', size=14)
    axs[1].set_xlabel('Longitude', size=14)
    axs[1].tick_params(axis='both', labelsize=12 )


##  States

In [ ]:
state_df = pd.read_csv('./data/state_sizes.csv')
# Remove two states that are barely in the study region
state_df = state_df.loc[~state_df['reg'].isin([11,15])]
state_shape = gpd.read_file('../accuracy/data/shapefiles/brazil_states_clip.shp')
make_map(state_df, state_shape, 'GEOCODIGO', 'State', log=False)

## Ecoregions

In [ ]:
eco_df = pd.read_csv('./data/eco_sizes.csv')
eco_shape = gpd.read_file('../accuracy/data/shapefiles/ecoregions_clip.shp')
make_map(eco_df, eco_shape, 'ECO_NUM', 'Ecoregion')

# Biomes


In [ ]:
biome_df = pd.read_csv('./data/biome_sizes.csv')
# Drop 9 and 14, very small/barely in study region
biome_df = biome_df.loc[~biome_df['reg'].isin([13,14])]
biome_shape = gpd.read_file('../accuracy/data/shapefiles/biome_clip.shp')
make_map(biome_df, biome_shape, 'BIOME', 'Biome', log=False)

## Watersheds


In [ ]:
water_df = pd.read_csv('./data/watersheds_sizes.csv')
# Drop smallest watershed
# water_df = water_df.loc[~water_df['reg'].isin([7238])]
water_shape = gpd.read_file('../accuracy/data/shapefiles/watersheds_4digit_clip.shp')
make_map(water_df, water_shape, 'NUNIVOTTO4', 'Watershed', log=True)

# Full Figure

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=[12,15])
make_map(state_df, state_shape, 'GEOCODIGO', 'State', log=False, axs=axs[0])
make_map(biome_df, biome_shape, 'BIOME', 'Biome', log=False, axs=axs[1])
make_map(water_df, water_shape, 'NUNIVOTTO4', 'Watershed', log=True, axs=axs[2])
fig.tight_layout() 
